In [ ]:
import os
from PyPDF2 import PdfMerger

pdf_files= ['Artificial Intelligence For Dummies.pdf','AI_12_probability.pdf','AI_13_Bayes_nets.pdf','AI_14_Markov_models.pdf','AI_1_Introduction.pdf'
            ,'AI_2_Agents.pdf','AI_3_Search.pdf','AI_6_CSP.pdf','AI_7_PL.pdf','AI_8_FOL.pdf','AI_book.pdf']

merger = PdfMerger()
for pdf in pdf_files:
    merger.append(pdf)

output_file = "gemeration_dataset.pdf"
merger.write(output_file)
merger.close()

print(f"merged_pdf_file is {output_file}")

import fitz 
from tqdm.auto import tqdm 
def text_formatter(text: str) -> str: 
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()

    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = [] 
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({"page_number": page_number - 41,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_setence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4, 
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path='gemeration_dataset.pdf')
pages_and_texts[:2]
import random

random.sample(pages_and_texts, k=3)
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()
df.describe().round(2)

from spacy.lang.en import English

nlp = English()

nlp.add_pipe("sentencizer")

doc = nlp("This is a sentence. This another sentence. I like elephants.")
assert len(list(doc.sents)) == 3

list(doc.sents)
pages_and_texts[600]

for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)

    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    item["page_sentence_count_spacy"] = len(item["sentences"])
random.sample(pages_and_texts, k=1)

df = pd.DataFrame(pages_and_texts)
df.describe().round(2)
num_sentence_chunk_size = 10

def split_list(input_list: list[str],
               slice_size: int=num_sentence_chunk_size) -> list[list[str]]:
    return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

test_list = list(range(25))
split_list(test_list)
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])
random.sample(pages_and_texts, k=1)
import re

pages_and_chunks = []
for item in tqdm(pages_and_texts): 
    for sentence_chunk in item["sentence_chunks"]: 
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)

        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 

        pages_and_chunks.append(chunk_dict) 

len(pages_and_chunks)
random.sample(pages_and_chunks, k=1)
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)
df.head()

min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]
random.sample(pages_and_chunks_over_min_token_len, k=1)
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cpu")

sentences = ["The Sentence Transformer library provides an easy way to create embeddings.",
             "Sentences can be embedded one by one or in a list.",
             "I like horses!"]

embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

for sentence, embedding in embeddings_dict.items():
    print(f"Sentence: {sentence}")
    print(f"Embedding: {embedding}")
    print("")
%%time

embedding_model.to("cuda")

for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

%%time

text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32, 
                                               convert_to_tensor=True) 

text_chunk_embeddings
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)
import pandas as pd
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()
import random

import torch
import numpy as np 
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")

text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape
text_chunks_and_embedding_df.head()
embeddings[0]
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device=device) query = "Machine Learning"
print(f"Query: {query}")

query_embedding = embedding_model.encode(query, convert_to_tensor=True)

from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product
larger_embeddings = torch.randn(100*embeddings.shape[0], 768).to(device)
print(f"Embeddings shape: {larger_embeddings.shape}")

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=larger_embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(larger_embeddings)} embeddings: {end_time-start_time:.5f} seconds.")
import textwrap

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)
print(f"Query: '{query}'\n")
print("Results:")
for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
    print(f"Score: {score:.4f}")
    print("Text:")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
    print(f"Page number: {pages_and_chunks[idx]['page_number']}")
    print("\n")
import fitz

pdf_path = "gemeration_dataset.pdf" 
doc = fitz.open(pdf_path)
page = doc.load_page(5 + 41) 

img = page.get_pixmap(dpi=300)

doc.close()

img_array = np.frombuffer(img.samples_mv, 
                          dtype=np.uint8).reshape((img.h, img.w, img.n))

import matplotlib.pyplot as plt
plt.figure(figsize=(13, 10))
plt.imshow(img_array)
plt.title(f"Query: '{query}' | Most relevant page:")
plt.axis('off') 
plt.show()
import torch

def dot_product(vector1, vector2):
    return torch.dot(vector1, vector2)

def cosine_similarity(vector1, vector2):
    dot_product = torch.dot(vector1, vector2)

    norm_vector1 = torch.sqrt(torch.sum(vector1**2))
    norm_vector2 = torch.sqrt(torch.sum(vector2**2))

    return dot_product / (norm_vector1 * norm_vector2)

vector1 = torch.tensor([1, 2, 3], dtype=torch.float32)
vector2 = torch.tensor([1, 2, 3], dtype=torch.float32)
vector3 = torch.tensor([4, 5, 6], dtype=torch.float32)
vector4 = torch.tensor([-1, -2, -3], dtype=torch.float32)

print("Dot product between vector1 and vector2:", dot_product(vector1, vector2))
print("Dot product between vector1 and vector3:", dot_product(vector1, vector3))
print("Dot product between vector1 and vector4:", dot_product(vector1, vector4))

print("Cosine similarity between vector1 and vector2:", cosine_similarity(vector1, vector2))
print("Cosine similarity between vector1 and vector3:", cosine_similarity(vector1, vector3))
print("Cosine similarity between vector1 and vector4:", cosine_similarity(vector1, vector4))
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
    """
    Embeds a query with model and returns top k scores and indices from embeddings.
    """

    query_embedding = model.encode(query, 
                                   convert_to_tensor=True) 

    start_time = timer()
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    end_time = timer()

    if print_time:
        print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

    scores, indices = torch.topk(input=dot_scores, 
                                 k=n_resources_to_return)

    return scores, indices

def print_top_results_and_scores(query: str,
                                 embeddings: torch.tensor,
                                 pages_and_chunks: list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):
    """
    Takes a query, retrieves most relevant resources and prints them out in descending order.

    Note: Requires pages_and_chunks to be formatted in a specific way (see above for reference).
    """
    
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings,
                                                  n_resources_to_return=n_resources_to_return)
    
    print(f"Query: {query}\n")
    print("Results:")
    for score, index in zip(scores, indices):
        print(f"Score: {score:.4f}")
        print_wrapped(pages_and_chunks[index]["sentence_chunk"])
        print(f"Page number: {pages_and_chunks[index]['page_number']}")
        print("\n")
query = "Reinforcement Learning"

scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices
print_top_results_and_scores(query=query,
                             embeddings=embeddings)

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Harsh0910/my_merged_model")
model = AutoModelForCausalLM.from_pretrained("Harsh0910/my_merged_model")
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(model)
def get_model_mem_size(model: torch.nn.Module):
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    model_mem_bytes = mem_params + mem_buffers 
    model_mem_mb = model_mem_bytes / (1024**2) 
    model_mem_gb = model_mem_bytes / (1024**3) 

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}

get_model_mem_size(model)
input_text = "What are steps of Reinforcement Learning?"

print(f"Input text:\n{input_text}")

dialogue_template = [
    {"role": "user",
     "content": input_text}
]

prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                       tokenize=False,
                                       add_generation_prompt=True)
print(f"\nPrompt (formatted):\n{prompt}")
print(prompt)
input_ids = tokenizer(prompt, return_tensors="pt")
print(f"Model input (tokenized):\n{input_ids}\n")

outputs = model.generate(**input_ids,
                             max_new_tokens=256) 
print(f"Model output (tokens):\n{outputs[0]}\n")
outputs_decoded = tokenizer.decode(outputs[0])
print(f"Model output (decoded):\n{outputs_decoded}\n")
print(f"Input text: {input_text}\n")
print(f"Output text:\n{outputs_decoded.replace(prompt, '').replace('<bos>', '').replace('<eos>', '')}")
gpt4_questions = [
    "What are the primary types of machine learning, and how do they differ in approach and application?",
    "How does natural language processing (NLP) enable machines to understand and generate human language?",
    "Describe the architecture of a neural network and its role in deep learning.",
    "What is reinforcement learning, and how is it applied in real-world scenarios?",
    "Explain the importance of data preprocessing in machine learning pipelines."
]

manual_questions = [
    "What is the Turing Test, and why is it significant in AI development?",
    "What are the steps of the backpropagation algorithm in training neural networks?",
    "How is overfitting detected, and what are strategies to mitigate it?",
    "What are the differences between precision, recall, and F1-score in evaluating model performance?",
    "Name three popular frameworks for building AI models and their primary features."
]
query_list = gpt4_questions + manual_questions
import random
query = random.choice(query_list)

print(f"Query: {query}")

scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
scores, indices
def prompt_formatter(query: str, 
                     context_items: list[dict]) -> str:
    """
    Augments query with text-based context from context_items.
    """
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])
    base_prompt = """Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
\nExample 1:
Query: What are the types of machine learning?
Answer: Machine learning can be categorized into three primary types: supervised learning, unsupervised learning, and reinforcement learning. Supervised learning involves training a model on labeled data, where the correct answers are provided, enabling the model to predict outcomes for new data. Unsupervised learning deals with unlabeled data, where the model identifies patterns or groupings without explicit guidance, such as clustering or dimensionality reduction. Reinforcement learning is based on training agents to make decisions by rewarding desirable behaviors and penalizing undesired ones, making it suitable for dynamic and interactive tasks like gaming and robotics.
\nExample 2:
Query: How do neural networks work?
Answer: Neural networks are computational models inspired by the human brain. They consist of layers of interconnected nodes, or neurons, which process data by assigning weights to inputs and applying an activation function. Data flows through input layers, hidden layers, and an output layer, where each connection's weight is adjusted during training using optimization techniques like backpropagation. This allows the network to learn complex patterns and relationships in the data, enabling tasks such as image recognition, natural language processing, and predictive analytics.
\nExample 3:
Query: What is transfer learning in AI?
Answer: Transfer learning in AI is a technique where a pre-trained model developed for one task is adapted to a different but related task. This approach leverages the knowledge already learned by the model, reducing the amount of data and computational resources needed for the new task. For instance, a neural network trained to recognize objects in images can be fine-tuned to identify specific types of objects, such as animals or vehicles. Transfer learning is especially valuable when working with limited labeled data for the target task, as it accelerates model development and improves accuracy.
\nNow use the following context items to answer the user query:
{context}
\nRelevant passages: <extract relevant passages from the context here>
User query: {query}
Answer:"""


    base_prompt = base_prompt.format(context=context, query=query)
    dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]

    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt
query = random.choice(query_list)
print(f"Query: {query}")

scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=embeddings)
    
context_items = [pages_and_chunks[i] for i in indices]

prompt = prompt_formatter(query=query,
                          context_items=context_items)
print(prompt)
%%time

input_ids = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**input_ids,
                             temperature=0.7, 
                             do_sample=True, 
                             max_new_tokens=256)

output_text = tokenizer.decode(outputs[0])

print(f"Query: {query}")
print(f"RAG answer:\n{output_text.replace(prompt, '')}")
def ask(query, 
        temperature=0.7,
        max_new_tokens=512,
        format_answer_text=True, 
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
    """
    
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings)
    
    context_items = [pages_and_chunks[i] for i in indices]

    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu() 
        
    prompt = prompt_formatter(query=query,
                              context_items=context_items)
    
    input_ids = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)
    
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "").replace("Sure, here is the answer to the user query:\n\n", "")

    if return_answer_only:
        return output_text
    
    return output_text, context_items
query = random.choice(query_list)
print(f"Query: {query}")

answer, context_items = ask(query=query, 
                            temperature=0.7,
                            max_new_tokens=512,
                            return_answer_only=False)

print(f"Answer:\n")
print_wrapped(answer)
print(f"Context items:")
context_items
from huggingface_hub import login
login()
import pickle

session_data = {
    "embeddings": embeddings.cpu(),  
    "pages_and_chunks": pages_and_chunks,
    "query": query,
    "answer": answer,
    "context_items": context_items
}

with open("session_data.pkl", "wb") as session_file:
    pickle.dump(session_data, session_file)

print("Session saved to 'session_data.pkl'")

from huggingface_hub import HfApi

api = HfApi()
repo_name = "my-session-data"  
repo_url = api.create_repo(repo_name, exist_ok=True)

print(f"Repository created: {repo_url}")

from huggingface_hub import HfApi, HfFolder

repo_id = "Harsh0910/my-session-data"  
file_path = "session_data.pkl"  

api = HfApi()
api.upload_file(
    path_or_fileobj=file_path,
    path_in_repo="session_data.pkl",  
    repo_id=repo_id,
    repo_type="model" 
)

print(f"File uploaded to Hugging Face Hub at: https://huggingface.co/{repo_id}")

from huggingface_hub import hf_hub_download
import pickle

file_path = hf_hub_download(repo_id="Harsh0910/my-session-data", filename="session_data.pkl")

with open(file_path, "rb") as session_file:
    session_data = pickle.load(session_file)

embeddings = session_data["embeddings"]
pages_and_chunks = session_data["pages_and_chunks"]
query = session_data["query"]
answer = session_data["answer"]
context_items = session_data["context_items"]

print("Session restored successfully from Hugging Face!")


merged_pdf_file is gemeration_dataset.pdf
